In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
from collections import Counter
from tensorflow.keras.preprocessing.image import img_to_array
import pyttsx3

try:
    model = tf.keras.models.load_model("EchoVision_1.h5", safe_mode=False)
    print(" Model loaded.")
except Exception as e:
    print( "model not loaded")
    exit()

IMG_SIZE = 224

class_labels = [
    "Someone's on a call right now.",
    "They're clapping — maybe for you!",
    "Looks like someone is out cycling.",
    "They're dancing — sounds like fun!",
    "They're drinking something — maybe thirsty.",
    "Someone's eating — smells good, doesn't it?",
    "There's a fight happening — stay alert.",
    "They're giving a warm hug.",
    "They're laughing — must be something funny!",
    "They're enjoying some music.",
    "They're running — maybe exercising.",
    "They're sitting down and relaxing.",
    "Someone's asleep — let's keep it quiet.",
    "They're texting on their phone."
]


framenum = 50
skipper = 3
timeout = 10 

engine = pyttsx3.init()
engine.setProperty('rate', 150)

def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame.astype("float32") / 255.0
    return img_to_array(frame)

def predict_action(frames):
    frames_array = np.array(frames)
    predictions = model.predict(frames_array, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    majority_vote = Counter(predicted_classes).most_common(1)[0][0]
    return class_labels[majority_vote]

cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("http://10.223.152.135:8080/video")

print("Press S to capture")

start_time = time.time()
frames = []
collected = 0
frame_count = 0
triggered = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (640, 480))
    cv2.imshow("Press 's' to Predict", resized_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        triggered = True
        print("capturing")
        break
    elif key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        exit()
    elif time.time() - start_time >= timeout:
        triggered = True
        break

if triggered:
    print("Please wait")
    while collected < framenum:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % skipper == 0:
            processed = preprocess_frame(frame)
            frames.append(processed)
            collected += 1
        frame_count += 1

cap.release()
cv2.destroyAllWindows()

if len(frames) == framenum:
    action = predict_action(frames)
    print(f" Predicted Action: {action}")
    engine.say(action)
    engine.runAndWait()
else:
    print("no enough frame.")
    engine.say("no enough frame.")
    engine.runAndWait()

ModuleNotFoundError: No module named 'cv2'